# Домашнее задание 2. Предобработка данных (42 балла)

## 1. Определитесь с тематикой вашего исследования. (3 балла)

Если нет своей идеи, то предлагается тема "Территории развития рэп-культуры (культуры хип-хопа) в Российской Федерации"

1. (1 балл) Впишите тему вашего исследования как комментарий.

2. (1 балл) Загрузите сессию VK API:

In [3]:
import vk  # Загружаем библиотеку vk

In [26]:
token = ""  # Сервисный ключ доступа, по завершении дз можно удалить для соблюдения конфиденциальности
session = vk.Session(access_token=token)  # Авторизация
vk_api = vk.API(session)  # Авторизация

3. (1 балл) Загрузите сессию с персональным ключом доступа с https://vkhost.github.io/

In [4]:
personal_token = ""  # Персональный ключ доступа, по завершении дз можно удалить для соблюдения конфиденциальности
session = vk.Session(access_token=personal_token)  # Авторизация
vk_personal_api = vk.API(session)  # Авторизация

## 2. Работа с пабликами по вашей тематике (6 баллов)

Обратите внимание — даже при использовании параметра offset для получения информации доступны только первые 1000 результатов.

In [8]:
def get_groups(query):
    first = vk_personal_api.groups.search(q=query, count=1000, type="group", v=5.124) 
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"]  # общее количество пабликов по вашему запросу
    return [(i['name'], i['id']) for i in data]

3. (3 балла) Найдите паблики, связанные с вашей темой. Используйте функцию `get_groups` как минимум трижды с разными ключевыми словами для поиска. Например:

`groups1 = get_groups("французский язык")`

`groups2 = get_groups("Франция")`

`groups3 = get_groups("франкофоны")`

In [7]:
# your code here

4. (2 балла) Объедините полученные результаты в `set`, чтобы избежать повторений. Сколько групп у вас получилось в итоге?

In [ ]:
# your code here

5. (1 балл) Сохраните полученные результаты в файл при помощи функции `save_groups_to_file`

In [9]:
def save_groups_to_file(filename, groups):
    with open(filename, 'w', encoding="utf-8") as f:
        for group_name, group_id in groups:
            f.write(f"{group_name}\t{group_id}\n")

In [ ]:
# your code here

## 3. Работа с конкретным пабликом (33 балла)

Выберите интересующий вас паблик (например, из списка пабликов предыдущего задания). В этой секции будем работать с ним.

### 3.1 Участники паблика (3 балла)

In [10]:
def get_members(groupid, fields=('sex', 'bdate', 'city', 'country', 'domain')):
    first = vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92)  # Первое выполнение метода
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 1000  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    for i in range(1, count+1):  
        data = data + vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92, offset=i*1000)["items"]
    return data

6. (1 балл) Используя функцию `get_members`, найдите всех пользователей интересующей вас группы. Результат сохраните в переменную `members`.

_Hint: на вход функции подается не название паблика, а его id или nickname из адресной строки. Например, участников паблика группы СБПЧ можно найти так:_


`members = get_members("sbp4band")`  

или так

`members = get_members("174327")` 

https://vk.com/sbp4band 

https://vk.com/club174327

In [ ]:
# your code here

7. (1 балл) Посчитайте общее количество участников:

In [ ]:
# your code here

8. (1 балл) Cохраните в файл данные об участниках при помощи функции `save_members_to_file`:

In [12]:
def save_members_to_file(filename, members):
    with open(filename, 'w', encoding='utf-8') as f:
        for member in members:
            _id = member["id"] # находим айди участника
            first_name = member["first_name"] # находим имя учатника
            last_name = member["last_name"] # находим фамилию учатника
            
            sex = member["sex"] # находим пол учатника (1 или 2)
            if int(sex) == 1:  # если пол это 1 
                sex = "F"  # обозначем женский пол как F
            elif int(sex) == 2:  # если пол это 2 
                sex = "M" # обозначем мужской пол как F
            else:  # если есть другой какой-то пол
                sex = "-"  # скорее всего нет, поэтому ставим прочерк
                
            bdate = member.get('bdate', "-")   # находим день рождения учатника
            if len(bdate.split(".")) == 3:  # нам нужен только год, если он есть
                bdate = bdate.split(".")[-1]  # то сохраняем год
            else:  # если год не указан, то возраст участника мы не узнаем
                bdate = "-"  # поэтому прочерк
                
            city = member.get('city', {}).get("title", "-")  # находим город участника
            country = member.get('country', {}).get("title", "-")  # находим страну участника
            f.write(f"{_id},\t{first_name}\t{last_name}\t{sex}\t{bdate}\t{city}\t{country}\n")  # сохраняем информацию в файл

In [13]:
# your code here

### 3.2. Работа с постами паблика (30 баллов)

9. (1 балл) Используя функцию, `get_posts`, выгрузите все посты интересующего вас паблика. Сохраните посты в переменную `posts`.

In [14]:
def get_posts(owner_id):
    first = vk_api.wall.get(domain=owner_id, count=100, filter="owner", v=5.124)
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 100  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    for i in range(1, count+1):  
        data += vk_api.wall.get(domain=owner_id, v=5.124, filter="owner", count=100, offset=i*100)["items"]
    return [i['text'] for i in data]

In [15]:
# your code here

10. (1 балл) Сколько постов у вас получилось?

In [16]:
# your code here

11. (3 балла) Напишите функцию `delete_newlines`, которая принимает на вход текст поста, заменяет в нем переходы на новую строку на пробелы и возвращает обратно текст поста (без переходов на новую строку).

In [32]:
# your code here

Если вы все сделали правильно, то следующая ячейка отработает без ошибок и сохранит посты без переходов на новую строку

In [31]:
posts = [delete_newlines(post) for post in posts]

In [37]:
with open("textss.txt", 'w', encoding="utf-8") as f:
    for post in posts:
        f.write(post+"\n")

12. (1 балл) Напечатайте 10 последних с конца списка постов, чтобы убедиться, что переходов на новую строку нет.

In [35]:
# your code here

13. (5 баллов) Напишите регулярное выражение для поиска ссылок в текстах постов.

Под ссылками понимается любая ссылка, она необязательно может начинаться с _https_, например:

http://www.vinylbox.ru/

https://sbp4band.ticketscloud.org

concert.ua/event/sbpch

bit.ly/2Sg0B1f

vk.cc/tickets

In [24]:
import re  # загружаем библиотеку для регулярок

In [88]:
# your regexp below
pattern_link = re.compile(r"your regexp")

14. (7 баллов) Проверим ваше регулярное выражение на тексте `test` (1 балл за каждую правильную ссылку):

In [87]:
test = '''На следующей неделе мы захватываем Екатеринбург!  2 марта, "Дом Печати"!  https://sbp4band.ticketscloud.org
Все спортсмены на месте! https://vk.com/sbp4_ekb 
Билеты: concert.ua/event/sbpch 
В Петербурге все билеты были проданы еще в пятницу, в Москве пока еще есть: vk.cc/7QWFXX 
Apple: http://goo.gl/rcmii8  Google: http://goo.gl/MzXj1C  Яндекс: https://vk.cc/7UrxWa'''

print(re.sub(pattern_link, "WEBLINK", test))

На следующей неделе мы захватываем Екатеринбург!  2 марта, "Дом Печати"!  WEBLINK
Все спортсмены на месте! WEBLINK 
Билеты: WEBLINK 
В Петербурге все билеты были проданы еще в пятницу, в Москве пока еще есть: WEBLINK 
Apple: WEBLINK


15. (1 балл) Очистим все тексты от ссылок:

In [89]:
def replace_weblinks(text):
    return re.sub(pattern_link, "WEBLINK", text)

In [90]:
posts = [replace_weblinks(post) for post in posts]

16. (3 балла) Напишите регулярное выражение для поиска внутренних ссылок вк на людей или группы в текстах постов. Например:
    
[id9062360|Арсений Горшенин]

([club185994949|СБПЧ / 9 ноября 2019 / Театро])

[club185957052|Новосибирск (Подземка])

*Hint: Действуйте по аналогии с регуляркой в 04_Regular_expressions_and_social_networks.ipynb*

In [98]:
# your regexp below
pattern_mention = re.compile(r"your regexp")

17. (4 балла) Проверим ваше регулярное выражение (1 балл за каждое правильно упоминание):

In [95]:
test2 = '''Журнал [club86423570|L'Officiel Russia] поцеловал нас в ушко.
7 ноября — [club185957052|Новосибирск (Подземка])  8 ноября — [club185959350|Красноярск (The Mods])
21 ноября — Пермь ([club185996153|СБПЧ / 21 ноября 2019 / М5])'''

print(re.sub(pattern_mention, "MENTION", test2))

Журнал MENTION поцеловал нас в ушко.
7 ноября — MENTION  8 ноября — MENTION
21 ноября — Пермь MENTION


18. (1 балл) Очистим все тексты от упоминаний:

In [97]:
def replace_mentions(text):
    return re.sub(pattern_mention, "MENTION", text)

In [ ]:
posts = [replace_mentions(post) for post in posts]

19. (3 балла) Сохраните очищенные от ссылок, уопминаний и новых строк тексты из `posts` в файл:

*Hint: Действуйте по аналогии с кодом в 04_Regular_expressions_and_social_networks.ipynb*

In [ ]:
# your code here